In [10]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import requests
import sqlite3

In [11]:
# Function to collect historical stock data
def get_stock_data(tickers, start_date, end_date):
    """
    Fetch historical stock data for multiple tickers from Yahoo Finance.

    Args:
        tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'MSFT']).
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: Stock price data for all tickers.
    """
    stock_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
    all_data = []
    for ticker in tickers:
        if ticker in stock_data.columns.levels[0]:
            df = stock_data[ticker].copy()
            df.reset_index(inplace=True)
            df['Ticker'] = ticker
            df['Date'] = pd.to_datetime(df['Date']).dt.date  # Convert to date format
            all_data.append(df)
    return pd.concat(all_data, ignore_index=True)


In [12]:
# Save data to SQLite database
def save_to_database(stock_data):
    """
    Save stock data and news data to SQLite database.

    Args:
        stock_data (pd.DataFrame): Stock price data.
        news_data (pd.DataFrame): News headlines data.
    """
    conn = sqlite3.connect('financial_data.db')

    # Save stock data
    stock_data.to_sql('stock_data', conn, if_exists='replace', index=False)
    conn.close()
    print("Data saved to SQLite database 'financial_data.db'.")

In [13]:
# Query and display data from SQLite database
def display_database_data():
    """
    Query and display data from the SQLite database.
    """
    conn = sqlite3.connect('financial_data.db')
    cursor = conn.cursor()

    # Query stock data
    print("Stock Data:")
    stock_query = "SELECT * FROM stock_data;"  # Adjust the LIMIT as needed
    stock_data = pd.read_sql_query(stock_query, conn)
    print(stock_data)

    
    conn.close()

In [14]:
# Query and load data from SQLite database
def load_data_from_database():
    """
    Load data from SQLite database into variables for ML operations.

    Returns:
        pd.DataFrame, pd.DataFrame: Stock data and news data.
    """
    conn = sqlite3.connect('financial_data.db')

    # Load stock data
    stock_data = pd.read_sql_query("SELECT * FROM stock_data;", conn)

    # Load news data
    #news_data = pd.read_sql_query("SELECT * FROM news_data;", conn)

    conn.close()
    print("Data loaded from SQLite database into variables.")
    return stock_data

In [15]:
 #Main workflow
if __name__ == "__main__":
    tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']  # Add more tickers as needed
    start_date = '2023-01-01'
    end_date = '2024-12-01'

    # Fetch stock data for multiple tickers
    stock_data = get_stock_data(tickers, start_date, end_date)

    
    # Save data to SQLite database
    save_to_database(stock_data)

    # Display data from the database
    display_database_data()
    # Load data into variables for ML operations
    stock_data= load_data_from_database()

[*********************100%***********************]  5 of 5 completed


Data saved to SQLite database 'financial_data.db'.
Stock Data:
            Date        Open        High         Low       Close     Volume  \
0     2023-01-03  128.924252  129.537796  122.877835  123.768471  112117500   
1     2023-01-04  125.569527  127.321112  123.778365  125.045044   89113600   
2     2023-01-05  125.807022  126.440361  123.461690  123.718979   80962700   
3     2023-01-06  124.698692  128.934144  123.590345  128.271118   87754700   
4     2023-01-09  129.112255  132.021662  128.538289  128.795578   70790800   
...          ...         ...         ...         ...         ...        ...   
2400  2024-11-22  341.089996  361.529999  337.700012  352.559998   89140700   
2401  2024-11-25  360.140015  361.929993  338.200012  338.589996   95890900   
2402  2024-11-26  341.000000  346.959991  335.660004  338.230011   62295900   
2403  2024-11-27  341.799988  342.549988  326.589996  332.890015   57896400   
2404  2024-11-29  336.079987  345.450012  334.649994  345.160004   3

In [16]:
stock_data

,Date,Open,High,Low,Close,Volume,Ticker
0,2023-01-03,128.924252,129.537796,122.877835,123.768471,112117500,AAPL
1,2023-01-04,125.569527,127.321112,123.778365,125.045044,89113600,AAPL
2,2023-01-05,125.807022,126.440361,123.461690,123.718979,80962700,AAPL
3,2023-01-06,124.698692,128.934144,123.590345,128.271118,87754700,AAPL
4,2023-01-09,129.112255,132.021662,128.538289,128.795578,70790800,AAPL
...,...,...,...,...,...,...,...
2400,2024-11-22,341.089996,361.529999,337.700012,352.559998,89140700,TSLA
2401,2024-11-25,360.140015,361.929993,338.200012,338.589996,95890900,TSLA
2402,2024-11-26,341.000000,346.959991,335.660004,338.230011,62295900,TSLA
2403,2024-11-27,341.799988,342.549988,326.589996,332.890015,57896400,TSLA


In [18]:
import plotly.graph_objects as go

def plot_interactive_stock_data(stock_data):
    """
    Create an interactive stock chart using Plotly with date interval controls.

    Args:
        stock_data (pd.DataFrame): Stock data with columns Date, Open, High, Low, Close, Volume, and Ticker.
    """
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    tickers = stock_data['Ticker'].unique()

    for ticker in tickers:
        ticker_data = stock_data[stock_data['Ticker'] == ticker]

        fig = go.Figure()

        # Candlestick chart
        fig.add_trace(go.Candlestick(
            x=ticker_data['Date'],
            open=ticker_data['Open'],
            high=ticker_data['High'],
            low=ticker_data['Low'],
            close=ticker_data['Close'],
            name='OHLC'
        ))

        # Volume as bar chart
        fig.add_trace(go.Bar(
            x=ticker_data['Date'],
            y=ticker_data['Volume'],
            name='Volume',
            marker_color='orange',
            opacity=0.6,
        ))

        # Add layout with range slider and range selector
        fig.update_layout(
            title=f"Stock Data for {ticker}",
            xaxis_title="Date",
            yaxis_title="Price",
            yaxis2=dict(
                title="Volume",
                overlaying="y",
                side="right",
                showgrid=False
            ),
            template="plotly_white",
            legend=dict(title="Metrics"),
            xaxis=dict(
                rangeselector=dict(
                    buttons=[
                        {"count": 1, "label": "1M", "step": "month", "stepmode": "backward"},
                        {"count": 3, "label": "3M", "step": "month", "stepmode": "backward"},
                        {"count": 6, "label": "6M", "step": "month", "stepmode": "backward"},
                        {"step": "all", "label": "All"}
                    ]
                ),
                rangeslider=dict(visible=True),  # Enable range slider
                type="date"  # Set x-axis type to date
            )
        )

        fig.show()

# Example usage
if __name__ == "__main__":
    # Assuming stock_data is loaded
    plot_interactive_stock_data(stock_data)
